CSV file

In [1]:
#Prepare CSV file

import pandas as pd
df = pd.read_csv('/Users/jafarabdurrahmaan/Dropbox/00_Uni/Masterthesis/A_Thesis/1_Components/13_Benchmark_Study_Components/3_KVS/CosmosDB/iter2.2/daily_dataset_small.csv')
df

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
0,MAC000131,2011-12-15,0.4850,0.432045,0.868,22,0.239146,9.505,0.072
1,MAC000131,2011-12-16,0.1415,0.296167,1.116,48,0.281471,14.216,0.031
2,MAC000131,2011-12-17,0.1015,0.189812,0.685,48,0.188405,9.111,0.064
3,MAC000131,2011-12-18,0.1140,0.218979,0.676,48,0.202919,10.511,0.065
4,MAC000131,2011-12-19,0.1910,0.325979,0.788,48,0.259205,15.647,0.066
...,...,...,...,...,...,...,...,...,...
4866,MAC000235,2014-01-28,0.1735,0.279000,1.114,48,0.241830,13.392,0.066
4867,MAC000235,2014-01-29,0.1865,0.320104,1.074,48,0.280150,15.365,0.068
4868,MAC000235,2014-01-30,0.3405,0.418750,1.277,48,0.293288,20.100,0.075
4869,MAC000235,2014-01-31,0.1635,0.325646,1.644,48,0.326863,15.631,0.058


In [2]:
import pandas as pd
df["id"] = df["LCLid"] + "_" + df["day"].astype(str)
columns = df.columns.tolist()
columns.insert(0, columns.pop(columns.index("id")))
df = df[columns]
df

,id,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
0,MAC000131_2011-12-15,MAC000131,2011-12-15,0.4850,0.432045,0.868,22,0.239146,9.505,0.072
1,MAC000131_2011-12-16,MAC000131,2011-12-16,0.1415,0.296167,1.116,48,0.281471,14.216,0.031
2,MAC000131_2011-12-17,MAC000131,2011-12-17,0.1015,0.189812,0.685,48,0.188405,9.111,0.064
3,MAC000131_2011-12-18,MAC000131,2011-12-18,0.1140,0.218979,0.676,48,0.202919,10.511,0.065
4,MAC000131_2011-12-19,MAC000131,2011-12-19,0.1910,0.325979,0.788,48,0.259205,15.647,0.066
...,...,...,...,...,...,...,...,...,...,...
4866,MAC000235_2014-01-28,MAC000235,2014-01-28,0.1735,0.279000,1.114,48,0.241830,13.392,0.066
4867,MAC000235_2014-01-29,MAC000235,2014-01-29,0.1865,0.320104,1.074,48,0.280150,15.365,0.068
4868,MAC000235_2014-01-30,MAC000235,2014-01-30,0.3405,0.418750,1.277,48,0.293288,20.100,0.075
4869,MAC000235_2014-01-31,MAC000235,2014-01-31,0.1635,0.325646,1.644,48,0.326863,15.631,0.058


In [9]:
import json
data = df.to_dict(orient='records')

# Save the data as a JSON file
with open('daily_dataset_small_kv.json', 'w') as f:
    json.dump(data, f, indent=4)

Connect to client

In [12]:
from azure.cosmos import CosmosClient, PartitionKey

In [13]:
endpoint = "https://smart-meter-benchmark.documents.azure.com:443/"
key = "HvrYXT5hnZmhRNJf2vJdXRy7Ms4YSKQE6lP77MAw4AES7N7sXLopbTyy1LoXYeBwRLgKEGms3S0JACDbKq9F3Q==;"
client = CosmosClient(endpoint, key)

Create database

In [14]:
from azure.cosmos import exceptions

database_name = "smart-meter-benchmark_mm"
try:
    database = client.create_database(id=database_name)
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(database=database_name)

Create container

In [15]:
from azure.cosmos import CosmosClient, PartitionKey, exceptions

# Container settings
container_id = "energy_mm"
partition_key_path = "/id_mm"

# Create the container
try:
    partition_key = PartitionKey(path=partition_key_path)
    container = database.create_container(
        id=container_id,
        partition_key=partition_key,
        offer_throughput=400,
    )
    print(f"Container created: {container.id}")

except exceptions.CosmosResourceExistsError:
    print("Container already exists.")

Container created: energy_mm


Insert data

In [22]:
import json
import time
from azure.cosmos import CosmosClient

# Set up Cosmos DB connection
endpoint = "https://smart-meter-benchmark.documents.azure.com:443/"
key = "HvrYXT5hnZmhRNJf2vJdXRy7Ms4YSKQE6lP77MAw4AES7N7sXLopbTyy1LoXYeBwRLgKEGms3S0JACDbKq9F3Q=="
client = CosmosClient(endpoint, key)

# Access the database and container
database_name = "smart-meter-benchmark_mm"
container_name = "energy_mm"
database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

# Read and insert JSON data
json_file = "/Users/jafarabdurrahmaan/Dropbox/00_Uni/Masterthesis/A_Thesis/1_Components/13_Benchmark_Study_Components/3_KVS/CosmosDB/iter2.2/daily_dataset_small_kv.json"

start_time = time.time()  # Start measuring time

with open(json_file, "r") as file:
    json_data = json.load(file)
    for item in json_data:
        # Specify the item's properties
        item_id = item["id"]
        lclid = item["LCLid"]
        day = item["day"]
        energy_median = float(item["energy_median"])
        energy_mean = float(item["energy_mean"])
        energy_max = float(item["energy_max"])
        energy_count = int(item["energy_count"])
        energy_std = float(item["energy_std"])
        energy_sum = float(item["energy_sum"])
        energy_min = float(item["energy_min"])

        # Create an item to be inserted into the container
        container_item = {
            "id": item_id,
            "LCLid": lclid,
            "day": day,
            "energy_median": energy_median,
            "energy_mean": energy_mean,
            "energy_max": energy_max,
            "energy_count": energy_count,
            "energy_std": energy_std,
            "energy_sum": energy_sum,
            "energy_min": energy_min,
            "partitionKey": "default"
        }

        # Insert the item into the container
        container.create_item(container_item)

end_time = time.time()  # Stop measuring time

execution_time = end_time - start_time
print(f"Data insertion took {execution_time:.2f} seconds")

Data insertion took 265.52 seconds


Simple Retrieval 1

In [35]:
from azure.cosmos import CosmosClient, exceptions

# Access the database and container
database_name = "smart-meter-benchmark_mm"
container_name = "energy_mm"
database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

# Specify the keys you want to query
keys = ["MAC000131_2012-06-01", "MAC000131_2012-06-02", "MAC000131_2012-06-03"]

# Construct the WHERE clause with multiple conditions
conditions = " OR ".join([f"c.id = '{key}'" for key in keys])
query = f"SELECT * FROM c WHERE {conditions}"

# Start measuring time
start_time = time.time()

try:
    # Query the container
    items = list(container.query_items(query, enable_cross_partition_query=True))
    
    # Process the query results
    for item in items:
        print(item)
except exceptions.CosmosHttpResponseError as e:
    # Handle any exceptions
    print(f"Error: {e.status_code} - {e.message}")

# Stop measuring time
end_time = time.time()  

execution_time = end_time - start_time
print(f"Query took {execution_time:.2f} seconds")

{'id': 'MAC000131_2012-06-01', 'LCLid': 'MAC000131', 'day': '2012-06-01', 'energy_median': 0.1105, 'energy_mean': 0.1203124999999999, 'energy_max': 0.312, 'energy_count': 48, 'energy_std': 0.0550555449697384, 'energy_sum': 5.774999999999998, 'energy_min': 0.064, 'partitionKey': 'default', '_rid': '54UyAOuB2GiqAAAAAAAAAA==', '_self': 'dbs/54UyAA==/colls/54UyAOuB2Gg=/docs/54UyAOuB2GiqAAAAAAAAAA==/', '_etag': '"33009d5d-0000-1000-0000-64a18f810000"', '_attachments': 'attachments/', '_ts': 1688309633}
{'id': 'MAC000131_2012-06-02', 'LCLid': 'MAC000131', 'day': '2012-06-02', 'energy_median': 0.092, 'energy_mean': 0.1137291666666667, 'energy_max': 0.299, 'energy_count': 48, 'energy_std': 0.0545717328369064, 'energy_sum': 5.459000000000001, 'energy_min': 0.065, 'partitionKey': 'default', '_rid': '54UyAOuB2GirAAAAAAAAAA==', '_self': 'dbs/54UyAA==/colls/54UyAOuB2Gg=/docs/54UyAOuB2GirAAAAAAAAAA==/', '_etag': '"33009f5d-0000-1000-0000-64a18f810000"', '_attachments': 'attachments/', '_ts': 1688309

Simple Retrieval 2

In [36]:
from azure.cosmos import CosmosClient, exceptions

# Access the database and container
database_name = "smart-meter-benchmark_mm"
container_name = "energy_mm"
database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

# Specify the keys you want to query
keys = ["MAC000131_2012-06-01", "MAC000131_2012-06-02", "MAC000131_2012-06-03"]

# Construct the WHERE clause with multiple conditions
conditions = " OR ".join([f"c.id = '{key}'" for key in keys])
query = f"SELECT * FROM c WHERE {conditions}"

# Start measuring time
start_time = time.time()

try:
    # Query the container
    items = list(container.query_items(query, enable_cross_partition_query=True))
    
    # Process the query results
    for item in items:
        print(item)
except exceptions.CosmosHttpResponseError as e:
    # Handle any exceptions
    print(f"Error: {e.status_code} - {e.message}")

# Stop measuring time
end_time = time.time()  

execution_time = end_time - start_time
print(f"Query took {execution_time:.2f} seconds")

{'id': 'MAC000131_2012-06-01', 'LCLid': 'MAC000131', 'day': '2012-06-01', 'energy_median': 0.1105, 'energy_mean': 0.1203124999999999, 'energy_max': 0.312, 'energy_count': 48, 'energy_std': 0.0550555449697384, 'energy_sum': 5.774999999999998, 'energy_min': 0.064, 'partitionKey': 'default', '_rid': '54UyAOuB2GiqAAAAAAAAAA==', '_self': 'dbs/54UyAA==/colls/54UyAOuB2Gg=/docs/54UyAOuB2GiqAAAAAAAAAA==/', '_etag': '"33009d5d-0000-1000-0000-64a18f810000"', '_attachments': 'attachments/', '_ts': 1688309633}
{'id': 'MAC000131_2012-06-02', 'LCLid': 'MAC000131', 'day': '2012-06-02', 'energy_median': 0.092, 'energy_mean': 0.1137291666666667, 'energy_max': 0.299, 'energy_count': 48, 'energy_std': 0.0545717328369064, 'energy_sum': 5.459000000000001, 'energy_min': 0.065, 'partitionKey': 'default', '_rid': '54UyAOuB2GirAAAAAAAAAA==', '_self': 'dbs/54UyAA==/colls/54UyAOuB2Gg=/docs/54UyAOuB2GirAAAAAAAAAA==/', '_etag': '"33009f5d-0000-1000-0000-64a18f810000"', '_attachments': 'attachments/', '_ts': 1688309

Simple Retrieval 3

In [37]:
from azure.cosmos import CosmosClient, exceptions

# Access the database and container
database_name = "smart-meter-benchmark_mm"
container_name = "energy_mm"
database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

# Specify the keys you want to query
keys = ["MAC000131_2012-06-01", "MAC000131_2012-06-02", "MAC000131_2012-06-03"]

# Construct the WHERE clause with multiple conditions
conditions = " OR ".join([f"c.id = '{key}'" for key in keys])
query = f"SELECT * FROM c WHERE {conditions}"

# Start measuring time
start_time = time.time()

try:
    # Query the container
    items = list(container.query_items(query, enable_cross_partition_query=True))
    
    # Process the query results
    for item in items:
        print(item)
except exceptions.CosmosHttpResponseError as e:
    # Handle any exceptions
    print(f"Error: {e.status_code} - {e.message}")

# Stop measuring time
end_time = time.time()  

execution_time = end_time - start_time
print(f"Query took {execution_time:.2f} seconds")

{'id': 'MAC000131_2012-06-01', 'LCLid': 'MAC000131', 'day': '2012-06-01', 'energy_median': 0.1105, 'energy_mean': 0.1203124999999999, 'energy_max': 0.312, 'energy_count': 48, 'energy_std': 0.0550555449697384, 'energy_sum': 5.774999999999998, 'energy_min': 0.064, 'partitionKey': 'default', '_rid': '54UyAOuB2GiqAAAAAAAAAA==', '_self': 'dbs/54UyAA==/colls/54UyAOuB2Gg=/docs/54UyAOuB2GiqAAAAAAAAAA==/', '_etag': '"33009d5d-0000-1000-0000-64a18f810000"', '_attachments': 'attachments/', '_ts': 1688309633}
{'id': 'MAC000131_2012-06-02', 'LCLid': 'MAC000131', 'day': '2012-06-02', 'energy_median': 0.092, 'energy_mean': 0.1137291666666667, 'energy_max': 0.299, 'energy_count': 48, 'energy_std': 0.0545717328369064, 'energy_sum': 5.459000000000001, 'energy_min': 0.065, 'partitionKey': 'default', '_rid': '54UyAOuB2GirAAAAAAAAAA==', '_self': 'dbs/54UyAA==/colls/54UyAOuB2Gg=/docs/54UyAOuB2GirAAAAAAAAAA==/', '_etag': '"33009f5d-0000-1000-0000-64a18f810000"', '_attachments': 'attachments/', '_ts': 1688309

In [38]:
print(f"Average Query Execution Time: {((0.05+0.05+0.05)/3)*1000} milliseconds")

Average Query Execution Time: 50.00000000000001 milliseconds


Metadata

In [28]:
from azure.cosmos import CosmosClient, exceptions

# Access the database and container
database_name = "smart-meter-benchmark_mm"
container_name = "energy_mm"
database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

# Define the query
query = "SELECT c.id, c._etag, c._ts FROM c WHERE c._ts > 1688200000"


# Start measuring time
start_time = time.time()

# Execute the query
try:
    items = list(container.query_items(query, enable_cross_partition_query=True))
    count = 0
    for item in items:
        print(item)
        count += 1
        if count >= 20:
            break
except exceptions.CosmosHttpResponseError as e:
    print("Error. {0}: {1}".format(e.status_code, e.message))
    
# Stop measuring time
end_time = time.time()  

execution_time = end_time - start_time
print(f"Query took {execution_time:.2f} seconds")

{'id': 'MAC000131_2011-12-15', '_etag': '"3300e45c-0000-1000-0000-64a18f790000"', '_ts': 1688309625}
{'id': 'MAC000131_2011-12-16', '_etag': '"3300e55c-0000-1000-0000-64a18f790000"', '_ts': 1688309625}
{'id': 'MAC000131_2011-12-17', '_etag': '"3300e65c-0000-1000-0000-64a18f790000"', '_ts': 1688309625}
{'id': 'MAC000131_2011-12-18', '_etag': '"3300e75c-0000-1000-0000-64a18f790000"', '_ts': 1688309625}
{'id': 'MAC000131_2011-12-19', '_etag': '"3300e95c-0000-1000-0000-64a18f790000"', '_ts': 1688309625}
{'id': 'MAC000131_2011-12-20', '_etag': '"3300ea5c-0000-1000-0000-64a18f790000"', '_ts': 1688309625}
{'id': 'MAC000131_2011-12-21', '_etag': '"3300eb5c-0000-1000-0000-64a18f790000"', '_ts': 1688309625}
{'id': 'MAC000131_2011-12-22', '_etag': '"3300ec5c-0000-1000-0000-64a18f790000"', '_ts': 1688309625}
{'id': 'MAC000131_2011-12-23', '_etag': '"3300ed5c-0000-1000-0000-64a18f790000"', '_ts': 1688309625}
{'id': 'MAC000131_2011-12-24', '_etag': '"3300ee5c-0000-1000-0000-64a18f790000"', '_ts': 16

Keys

In [29]:
from azure.cosmos import CosmosClient, exceptions

# Access the database and container
database_name = "smart-meter-benchmark_mm"
container_name = "energy_mm"
database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

# Define the query
query = "SELECT c.id FROM c"

# Start measuring time
start_time = time.time()

# Execute the query
try:
    items = list(container.query_items(query, enable_cross_partition_query=True))
    count = 0
    for item in items:
        print(item)
        count += 1
        if count >= 5:
            break
except exceptions.CosmosHttpResponseError as e:
    print("Error. {0}: {1}".format(e.status_code, e.message))
    
# Stop measuring time
end_time = time.time()  

execution_time = end_time - start_time
print(f"Query took {execution_time:.2f} seconds")

{'id': 'MAC000131_2011-12-15'}
{'id': 'MAC000131_2011-12-16'}
{'id': 'MAC000131_2011-12-17'}
{'id': 'MAC000131_2011-12-18'}
{'id': 'MAC000131_2011-12-19'}
Query took 0.45 seconds
